In [1]:
!pip install pennylane pennylane-qiskit tensorflow qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.6 

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

import tensorflow as tf
import warnings
from google.colab import drive
drive.mount('/content/drive')

# Set TensorFlow logging level to ERROR
tf.get_logger().setLevel('ERROR')

# Suppress specific warnings from TensorFlow
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow')

# Load and preprocess the dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final_Dataset.csv')

# Select features and target column
X = data.drop(columns=['Label', 'Label.1'])
y = data['Label']

# Encode labels to binary format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply PCA for dimensionality reduction, retaining 20 components
pca = PCA(n_components=20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# **Reshape labels** to match the model output shape
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pennylane as qml
import tensorflow as tf

# Set up a quantum device with 20 qubits
n_qubits = 20
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum convolutional layer
def qconv_layer(x):
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))
    for i in range(n_qubits):
        qml.RY(0.25 * 3.14159, wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])

# Quantum node ensuring only real-valued outputs
@qml.qnode(dev, interface="tf")
def quantum_circuit(inputs):
    qconv_layer(inputs)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define QuantumLayer with an explicit output shape
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()
        # Initialize a trainable weight for each qubit
        self.weight = tf.Variable(tf.random.normal([n_qubits]), trainable=True)

    def call(self, inputs):
        # Scale inputs to match the number of qubits
        scaled_inputs = inputs * tf.reshape(self.weight, (1, -1))

        # Apply the quantum circuit
        quantum_outputs = tf.vectorized_map(quantum_circuit, scaled_inputs)

        # Reshape to ensure a defined output shape
        quantum_outputs = tf.reshape(quantum_outputs, (-1, n_qubits))

        return tf.cast(quantum_outputs, dtype=tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], n_qubits)


In [20]:
tf.config.run_functions_eagerly(False)

from tensorflow.keras.layers import Input

model = tf.keras.models.Sequential([
    Input(shape=(20,)),                        # Explicit input layer with shape (20,)
    QuantumLayer(),                             # Quantum feature extraction layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import mixed_precision

# Enable mixed precision if on a compatible GPU
mixed_precision.set_global_policy('mixed_float16')

# Define early stopping to save time on unnecessary training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,  # Stop if validation loss doesn’t improve for 3 epochs
    restore_best_weights=True
)

# Train the model with optimized settings
history = model.fit(
    X_train_pca,
    y_train,
    epochs=10,               # Fewer epochs for quicker results
    batch_size=16,           # Smaller batch size for faster updates
    validation_data=(X_test_pca, y_test),
    callbacks=[early_stopping],
    verbose=1                # Reduced verbosity
)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_pca, y_test, verbose=1)
print(f"Test accuracy: {accuracy:.4f}")


Epoch 1/10
 70/529 [==>...........................] - ETA: 5:28:50 - loss: 0.8836 - accuracy: 0.2696